# Implementera ett expertssystem för djur

Ett exempel från [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

I detta exempel ska vi implementera ett enkelt kunskapsbaserat system för att bestämma ett djur utifrån några fysiska egenskaper. Systemet kan representeras av följande AND-OR-träd (detta är en del av hela trädet, vi kan enkelt lägga till fler regler):

![](../../../../../../translated_images/sv/AND-OR-Tree.5592d2c70187f283.webp)


## Vårt eget expertsystemskal med bakåtlogik

Låt oss försöka definiera ett enkelt språk för kunskapsrepresentation baserat på produktionsregler. Vi kommer att använda Python-klasser som nyckelord för att definiera regler. Det finns i princip 3 typer av klasser:
* `Ask` representerar en fråga som behöver ställas till användaren. Den innehåller mängden möjliga svar.
* `If` representerar en regel, och är bara ett syntaktiskt socker för att lagra innehållet i regeln
* `AND`/`OR` är klasser för att representera OCH/ELLER-grenar i trädet. De lagrar bara listan med argument inuti. För att förenkla koden är all funktionalitet definierad i basklassen `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

I vårt system skulle arbetsminnet innehålla listan över **fakta** som **attribut-värde-par**. Kunskapsbasen kan definieras som ett stort ordbok som kopplar åtgärder (nya fakta som ska infogas i arbetsminnet) till villkor, uttryckta som OCH-ELLER-uttryck. Dessutom kan vissa fakta `Ask`as.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

För att utföra den bakåtledda slutledningen definierar vi klassen `Knowledgebase`. Den kommer att innehålla:
* Arbetsminne `memory` - en ordbok som kopplar attribut till värden
* Regler i kunskapsbasen `rules` i det format som definierats ovan

Två huvudmetoder är:
* `get` för att erhålla värdet på ett attribut och utföra slutledning vid behov. Till exempel, `get('color')` skulle hämta värdet på en färgslot (den kommer att fråga vid behov, och lagra värdet för senare användning i arbetsminnet). Om vi frågar `get('color:blue')`, kommer den att fråga efter färgen, och sedan returnera värdet `y`/`n` beroende på färgen.
* `eval` utför den faktiska slutledningen, dvs. traverserar AND/OR-trädet, utvärderar delmål, etc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Nu ska vi definiera vår djurkunskapsbas och genomföra konsultationen. Observera att detta samtal kommer att ställa frågor till dig. Du kan svara genom att skriva `y`/`n` för ja-nej-frågor, eller genom att ange nummer (0..N) för frågor med längre flervalsalternativ.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Använda Experta för Framåtbunden Slutledning

I nästa exempel kommer vi att försöka implementera framåtbunden slutledning med hjälp av ett av biblioteken för kunskapsrepresentation, [Experta](https://github.com/nilp0inter/experta). **Experta** är ett bibliotek för att skapa framåtbundna slutledning system i Python, som är utformat för att likna det klassiska gamla systemet [CLIPS](http://www.clipsrules.net/index.html).

Vi skulle också kunna ha implementerat framåtbunden kedjning själva utan större problem, men naiva implementationer är vanligtvis inte särskilt effektiva. För mer effektiv regelmatchning används en speciell algoritm [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Vi definierar vårt system som en klass som är en subklass till `KnowledgeEngine`. Varje regel definieras av en separat funktion med `@Rule`-annotering, som specificerar när regeln ska aktiveras. Inuti regeln kan vi lägga till nya fakta med funktionen `declare`, och att lägga till dessa fakta kommer att resultera i att fler regler anropas av den framåtriktade inferensmotorn.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

När vi har definierat en kunskapsbas fyller vi vårt arbetsminne med några initiala fakta och anropar sedan `run()`-metoden för att utföra inferensen. Du kan se som ett resultat att nya härledda fakta läggs till i arbetsminnet, inklusive det slutliga faktumet om djuret (om vi har ställt in alla initiala fakta korrekt).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av den AI-baserade översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Vi strävar efter noggrannhet, men var medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess modersmål bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår till följd av användningen av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
